In [1]:
import numpy as np
import tensorflow as tf
import subprocess

import subprocess, os
import concurrent.futures, json, threading, psutil, time

import pandas as pd


p = 21888242871839275222246405745257275088548364400416034343698204186575808495617

2024-02-06 17:57:43.437512: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 17:57:43.470025: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 17:57:43.470052: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 17:57:43.470754: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 17:57:43.475462: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and flatten the images
train_images_tf = train_images.reshape((-1, 28*28)) / 255.0
test_images_tf = test_images.reshape((-1, 28*28)) / 255.0

# Resize for 14 * 14 images
train_images_tf_reshaped = tf.reshape(train_images_tf, [-1, 28, 28, 1])  # Reshape to [num_samples, height, width, channels]
test_images_tf_reshaped = tf.reshape(test_images_tf, [-1, 28, 28, 1])

# Downsample images
train_images_tf_downsampled = tf.image.resize(train_images_tf_reshaped, [14, 14], method='bilinear')
test_images_tf_downsampled = tf.image.resize(test_images_tf_reshaped, [14, 14], method='bilinear')

# Flatten the images back to [num_samples, 14*14]
train_images_tf_downsampled = tf.reshape(train_images_tf_downsampled, [-1, 14*14])
test_images_tf_downsampled = tf.reshape(test_images_tf_downsampled, [-1, 14*14])

2024-02-05 12:17:31.933254: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-05 12:17:31.959199: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-05 12:17:31.959341: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [31]:
def get_predictions_tf(model, test_images, batch_size=256):
    predictions = []
    for i in range(0, len(test_images), batch_size):
        batch = test_images[i:i+batch_size]
        pred = model.predict(batch)
        predictions.extend(np.argmax(pred, axis=1))
    return predictions

In [32]:
def transfer_weights(layers, model, scalar = 36):
    weights = []
    biases = []
    for ind in range(len(layers)-1):
        w = [[int(model.weights[ind * 2].numpy()[i][j]*10**scalar) for j in range(layers[ind+1])] for i in range(layers[ind])]
        b = [int(model.weights[ind * 2 + 1].numpy()[i]*10**(scalar * 2)) for i in range(layers[ind+1])]
        #b = [0 for i in range(layers[ind+1])]
        weights.append(w)
        biases.append(b)

    return weights, biases

In [33]:
def relu_mod(x):
    return x if x < p // 2 else 0

def DenseInt(nInputs, nOutputs, n, input, weights, bias):
    #print (len(input), nInputs)
    
    Input = [str(input[i] % p) for i in range(nInputs)]
    Weights = [[str(weights[i][j] % p) for j in range(nOutputs)] for i in range(nInputs)]
    Bias = [str(bias[i] % p) for i in range(nOutputs)]
    
    out = [0 for _ in range(nOutputs)]
    remainder = [None for _ in range(nOutputs)]
    
    for j in range(nOutputs):
        for i in range(nInputs):
            out[j] += input[i] * weights[i][j]
        out[j] += bias[j]

        remainder[j] = str(out[j] % n)
        out[j] = out[j] // n % p
        
    return Input, Weights, Bias, out, remainder

In [34]:
def prepare_input_json(layers, weights, biases, x_in, scalar=36, relu = False):
    relu_outs = []
    dense_weights = []
    dense_biases = []
    dense_outs = []
    dense_remainders = []
    x_ins = []

    out = x_in
    for ind in range(len(weights)):
        nInputs = layers[ind]
        nOutputs = layers[ind + 1]
        #print (nInputs, nOutputs)
        x_in, w, b, out, rem = DenseInt(nInputs, nOutputs, 10 ** scalar, 
                                     out, weights[ind], biases[ind])
        
        dense_outs.append(out)
        if relu:
            out = [x if x < p//2 else 0 for x in out]
            relu_outs.append([str(x) if x !=0 else 0 for x in out ])

        #print (out)
        dense_weights.append(w)
        dense_biases.append(b)
        
        dense_remainders.append(rem)
        x_ins.append(x_in)

    
    dense_outs = [[str(x) for x in sub] for sub in dense_outs]
        
    return x_ins[0], dense_weights, dense_biases, dense_outs, dense_remainders, relu_outs, np.argmax(out)

In [35]:
def monitor_memory(pid, freq = 0.01):
    p = psutil.Process(pid)
    max_memory = 0
    while True:
        try:
            mem = p.memory_info().rss / (1024 * 1024)
            max_memory = max(max_memory, mem)
        except psutil.NoSuchProcess:
            break  # Process has finished
        time.sleep(freq)  # Poll every second
        
    #print(f"Maximum memory used: {max_memory} MB")
    return max_memory

def execute_and_monitor(command, show = False):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(monitor_memory, process.pid)
        stdout, stderr = process.communicate()
        max_memory = future.result()
    if show:
        print(f"Maximum memory used: {max_memory} MB")
    return stdout, stderr, max_memory

In [9]:
def benchmark(test_images, predictions, weights, biases, layers, tmp_folder, input_path, zkey, veri_key):
    loss = 0

    target_circom = "_".join(str(x) for x in layers) + '.circom'

    json_folder = tmp_folder + target_circom[:-7] + "_js/"
    wit_json_file = json_folder + "generate_witness.js"
    wasm_file = json_folder + target_circom[:-7] + ".wasm"
    input_path = tmp_folder + "input.json"
    wit_file = tmp_folder + "witness.wtns"

    mem_usage = []
    time_cost = []
    benchmark_start_time = time.time()

    for i in range(len(test_images)):
        cost = 0
        X = test_images[i:i+1]
        start_time = time.time()
        X_in = [int(x*1e36) for x in X[0]]
        x_in, dense_weights, dense_biases, dense_outs, dense_remainders, relu_outs, pred = prepare_input_json(layers, weights, biases, X_in, scalar=36, relu=True)

        in_json = {
            "in": x_in,
            "Dense32weights": dense_weights[0],
            "Dense32bias": dense_biases[0],
            "Dense32out": dense_outs[0],
            "Dense32remainder": dense_remainders[0],
            "ReLUout": relu_outs[0], 
            "Dense21weights": dense_weights[1],
            "Dense21bias": dense_biases[1],
            "Dense21out": dense_outs[1],
            "Dense21remainder": dense_remainders[1]
        }

        with open(input_path, "w") as f:
            json.dump(in_json, f)

        if pred != predictions[i]:
            loss += 1
            print ("Loss happens on index", i)

        commands = [['node', wit_json_file, wasm_file, input_path, wit_file],
                    ['snarkjs', 'groth16', 'prove',zkey, wit_file, tmp_folder+'proof.json', tmp_folder+'public.json']]
                    #['snarkjs', 'groth16', 'verify',veri_key, tmp_folder+'public.json', tmp_folder+'proof.json']]

        for command in commands:
            _, _, usage = execute_and_monitor(command)
            cost += usage
        #print ("stdout:", stdout)
            
        mem_usage.append(cost)
        time_cost.append(time.time() - start_time)
    
    print ("Total time:", time.time() - benchmark_start_time)
    return loss, mem_usage, time_cost


In [10]:
def benchmark_(test_images, predictions, weights, biases, layers, tmp_folder, input_path, zkey, veri_key):
    loss = 0

    target_circom = "_".join(str(x) for x in layers) + '.circom'

    json_folder = tmp_folder + target_circom[:-7] + "_js/"
    wit_json_file = json_folder + "generate_witness.js"
    wasm_file = json_folder + target_circom[:-7] + ".wasm"
    input_path = tmp_folder + "input.json"
    wit_file = tmp_folder + "witness.wtns"

    mem_usage = []
    time_cost = []
    benchmark_start_time = time.time()

    for i in range(len(test_images)):
        cost = 0
        X = test_images[i:i+1]
        start_time = time.time()
        X_in = [int(x*1e36) for x in X[0]]
        x_in, dense_weights, dense_biases, dense_outs, dense_remainders, relu_outs, pred = prepare_input_json(layers, weights, biases, X_in, scalar=36, relu=True)


        in_json = {
            "in": x_in,
            "Dense32weights": dense_weights[0],
            "Dense32bias": dense_biases[0],
            "Dense32out": dense_outs[0],
            "Dense32remainder": dense_remainders[0],
            "ReLUout": relu_outs[0], 
            "Dense21weights": dense_weights[1],
            "Dense21bias": dense_biases[1],
            "Dense21out": dense_outs[1],
            "Dense21remainder": dense_remainders[1],
            "ReLUout2": relu_outs[1],
            "Dense10weights": dense_weights[2],
            "Dense10bias": dense_biases[2],
            "Dense10out": dense_outs[2],
            "Dense10remainder": dense_remainders[2]
        }

        with open(input_path, "w") as f:
            json.dump(in_json, f)

        if pred != predictions[i]:
            loss += 1
            print ("Loss happens on index", i)

        commands = [['node', wit_json_file, wasm_file, input_path, wit_file],
                    ['snarkjs', 'groth16', 'prove',zkey, wit_file, tmp_folder+'proof.json', tmp_folder+'public.json']]
                    #['snarkjs', 'groth16', 'verify',veri_key, tmp_folder+'public.json', tmp_folder+'proof.json']]

        for command in commands:
            _, _, usage = execute_and_monitor(command)
            cost += usage
        #print ("stdout:", stdout)
            
        mem_usage.append(cost)
        time_cost.append(time.time() - start_time)
    
    print ("Total time:", time.time() - benchmark_start_time)
    return loss, mem_usage, time_cost

# DNN

## For 196_25_10 DNN Model

In [9]:
layers = [196,25, 10]

inputs = tf.keras.layers.Input(shape=(layers[0],))
out = tf.keras.layers.Dense(layers[1], activation = 'relu')(inputs)
out = tf.keras.layers.Dense(layers[2])(out)

model = tf.keras.Model(inputs, out)

In [10]:
model_path = "../../models/"
arch_folder = "input-dense-dense/"
model_name = "196_25_10.h5"
model_in_path = model_path+arch_folder+model_name

model.load_weights(model_in_path)
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 196)]             0         
                                                                 
 dense (Dense)               (None, 25)                4925      
                                                                 
 dense_1 (Dense)             (None, 10)                260       
                                                                 
Total params: 5185 (20.25 KB)
Trainable params: 5185 (20.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### Test Accuracy

In [11]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images_tf_downsampled, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

2024-02-03 06:26:24.141787: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


313/313 - 1s - loss: 0.8117 - accuracy: 0.9541 - 748ms/epoch - 2ms/step

Test accuracy: 0.9541000127792358


In [12]:
predictions_tf = get_predictions_tf(model, test_images_tf_downsampled)

1/1 [==============================] - 0s 21ms/step


### Generate Input Json

In [15]:
weights, biases = transfer_weights(layers, model, 36)

In [16]:
# Generate sample input
X = test_images_tf_downsampled[0:1]
#X = np.random.rand(1,layers[0])


X_in = [int(x*1e36) for x in X[0]]

In [17]:
x_in, dense_weights, dense_biases, dense_outs, dense_remainders, relu_outs, pred = prepare_input_json(layers, weights, biases, X_in, scalar=36, relu=True)
#np.argmax(dense_outs[-1]), relu_outs

In [18]:
in_json = {
    "in": x_in,
    "Dense32weights": dense_weights[0],
    "Dense32bias": dense_biases[0],
    "Dense32out": dense_outs[0],
    "Dense32remainder": dense_remainders[0],
    "ReLUout": relu_outs[0], 
    "Dense21weights": dense_weights[1],
    "Dense21bias": dense_biases[1],
    "Dense21out": dense_outs[1],
    "Dense21remainder": dense_remainders[1]
}


with open("./tmp/input.json", "w") as f:
    json.dump(in_json, f)

In [19]:
model.predict(X), np.argmax(model.predict(X)), pred

1/1 [==============================] - 0s 9ms/step


(array([[ -3.940578  ,  -7.802052  ,   0.01312168,   1.882688  ,
          -7.3138003 ,  -0.84990066, -10.740956  ,   7.80357   ,
          -2.9416392 ,  -0.21228142]], dtype=float32),
 7,
 7)

### Benchmarking Proof Gen

In [171]:
test_size = 2500
loss, mem_usage, time_cost = benchmark(test_images_tf[:test_size], predictions_tf[:test_size], weights, biases, layers, "./tmp/", "./tmp/input.json", zkey_1, veri_key)

Total time: 2924.856101989746


In [173]:
csv_path = '../../benchmarks/benchmark_results.csv'

columns = ['Framework', 'Architecture', '# Layers', '# Parameters', 'Testing Size', 'Accuracy Loss (%)', 
           'Avg Memory Usage (MB)', 'Std Memory Usage', 'Avg Proving Time (s)', 'Std Proving Time']

# Check if the CSV file exists
if not os.path.isfile(csv_path):
    # Create a DataFrame with the specified columns
    df = pd.DataFrame(columns=columns)
    # Save the DataFrame as a CSV file
    df.to_csv(csv_path, index=False)
else:
    print(f"File '{csv_path}' already exists.")

df = pd.read_csv(csv_path)

File '../../benchmarks/benchmark_results.csv' already exists.


In [174]:
new_row = {
    'Framework': ['circomlib-ml (tensorflow)'],
    'Architecture': ['Input-Dense-Dense (196 * 25 * 10'],
    '# Layers': [3],
    '# Parameters': [5185],
    'Testing Size': [test_size],
    'Accuracy Loss (%)': [loss/test_size * 100],
    'Avg Memory Usage (MB)': [sum(mem_usage) / len(mem_usage)],
    'Std Memory Usage': [pd.Series(mem_usage).std()],
    'Avg Proving Time (s)': [sum(time_cost) / len(time_cost)],
    'Std Proving Time': [pd.Series(time_cost).std()]
}

new_row_df = pd.DataFrame(new_row)


In [176]:
df = pd.concat([df, new_row_df], ignore_index=True)
df.to_csv(csv_path, index=False)
df

,Framework,Architecture,# Layers,# Parameters,Testing Size,Accuracy Loss (%),Avg Memory Usage (MB),Std Memory Usage,Avg Proving Time (s),Std Proving Time
0,opml (pytorch),Input-Dense-Dense (784 * 56 * 10),3,44543,250,0.40,88.998094,2.285579,3.655122,0.440126
1,opml (pytorch),"Input-Dense-Dense (784 * 56 * 10, w/ relu)",3,44543,1000,20.90,89.122078,2.247846,3.664727,0.433921
2,opml (pytorch),Input-Dense-Dense (784 * 56 * 10),3,44543,2500,0.72,89.120883,2.254392,3.609974,0.421732
3,opml (pytorch),Input-Dense-Dense (196 * 25 * 10,3,5185,5000,3.52,74.351948,1.419725,0.809440,0.074357
4,circomlib-ml (tensorflow),Input-Dense-Dense (196 * 25 * 10,3,5185,2500,0.00,998.735128,8.040170,1.169482,0.108491


## For 784_56_10 DNN Model

In [20]:
layers = [784, 56, 10]

inputs = tf.keras.layers.Input(shape=(layers[0],))
out = tf.keras.layers.Dense(layers[1], activation = 'relu')(inputs)
out = tf.keras.layers.Dense(layers[2])(out)

model = tf.keras.Model(inputs, out)

In [21]:
model_path = "../../models/"
arch_folder = "input-dense-dense/"
model_name = "784_56_10.h5"
model_in_path = model_path+arch_folder+model_name

model.load_weights(model_in_path)
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense_2 (Dense)             (None, 56)                43960     
                                                                 
 dense_3 (Dense)             (None, 10)                570       
                                                                 
Total params: 44530 (173.95 KB)
Trainable params: 44530 (173.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images_tf, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 0s - loss: 0.5807 - accuracy: 0.9740 - 348ms/epoch - 1ms/step

Test accuracy: 0.9739999771118164


In [23]:
predictions_tf = get_predictions_tf(model, test_images_tf)

1/1 [==============================] - 0s 31ms/step


### Generate Input Json with Sampled Img

In [24]:
weights, biases = transfer_weights(layers, model, 36)

In [28]:
X = test_images_tf[1:2]

X_in = [int(x*1e36) for x in X[0]]

In [29]:
x_in, dense_weights, dense_biases, dense_outs, dense_remainders, relu_outs, pred = prepare_input_json(layers, weights, biases, X_in, scalar=36, relu=True)


In [30]:
import json

in_json = {
    "in": x_in,
    "Dense32weights": dense_weights[0],
    "Dense32bias": dense_biases[0],
    "Dense32out": dense_outs[0],
    "Dense32remainder": dense_remainders[0],
    "ReLUout": relu_outs[0], 
    "Dense21weights": dense_weights[1],
    "Dense21bias": dense_biases[1],
    "Dense21out": dense_outs[1],
    "Dense21remainder": dense_remainders[1]
}


with open("./tmp/input.json", "w") as f:
    json.dump(in_json, f)

In [31]:
model.predict(X), np.argmax(model.predict(X)), pred

1/1 [==============================] - 0s 14ms/step


(array([[ -2.9282103,   6.6508713,  11.407956 ,   1.3294847, -21.93504  ,
           1.6703438,  -1.5391957, -17.08455  ,   2.6426253, -19.287441 ]],
       dtype=float32),
 2,
 2)

### Benchmark

In [39]:
test_size = 2500
loss, mem_usage, time_cost = benchmark(test_images_tf[:test_size], predictions_tf[:test_size], weights, biases, layers, "./tmp/", "./tmp/input.json", zkey_1, veri_key)

Loss happens on index 1607
Total time: 5878.2243745327


In [40]:
new_row = {
    'Framework': ['circomlib-ml (tensorflow)'],
    'Architecture': ['Input-Dense-Dense (784x56x10'],
    '# Layers': [3],
    '# Parameters': [44543],
    'Testing Size': [test_size],
    'Accuracy Loss (%)': [loss/test_size * 100],
    'Avg Memory Usage (MB)': [sum(mem_usage) / len(mem_usage)],
    'Std Memory Usage': [pd.Series(mem_usage).std()],
    'Avg Proving Time (s)': [sum(time_cost) / len(time_cost)],
    'Std Proving Time': [pd.Series(time_cost).std()]
}

new_row_df = pd.DataFrame(new_row)

In [42]:
csv_path = '../../benchmarks/benchmark_results.csv'

columns = ['Framework', 'Architecture', '# Layers', '# Parameters', 'Testing Size', 'Accuracy Loss (%)', 
           'Avg Memory Usage (MB)', 'Std Memory Usage', 'Avg Proving Time (s)', 'Std Proving Time']

# Check if the CSV file exists
if not os.path.isfile(csv_path):
    # Create a DataFrame with the specified columns
    df = pd.DataFrame(columns=columns)
    # Save the DataFrame as a CSV file
    df.to_csv(csv_path, index=False)
else:
    print(f"File '{csv_path}' already exists.")

df = pd.read_csv(csv_path)

File '../../benchmarks/benchmark_results.csv' already exists.


In [43]:
df = pd.concat([df, new_row_df], ignore_index=True)
df.to_csv(csv_path, index=False)
df

,Framework,Architecture,# Layers,# Parameters,Testing Size,Accuracy Loss (%),Avg Memory Usage (MB),Std Memory Usage,Avg Proving Time (s),Std Proving Time
0,opml (pytorch),Input-Dense-Dense (784x56x10),3,44543,2500,0.72,89.120883,2.254392,3.609974,0.421732
1,opml (pytorch),Input-Dense-Dense (196x25x10),3,5185,5000,3.52,74.351948,1.419725,0.809440,0.074357
2,circomlib-ml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,2500,0.00,998.735128,8.040170,1.169482,0.108491
3,zkml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,1000,0.10,2334.295629,12.335301,21.263430,0.294183
4,zkml (tensorflow),Input-Dense-Dense (784x56x10),3,44543,1000,0.00,2357.363156,11.750076,22.012543,0.317096
5,circomlib-ml (tensorflow),Input-Dense-Dense (784x56x10,3,44543,2500,0.04,2328.322411,34.436486,2.351284,0.147113


## For 196_24_14_10 DNN Model

In [11]:
layers = [196,24,14,10]

inputs = tf.keras.layers.Input(shape=(layers[0],))
out = tf.keras.layers.Dense(layers[1], activation = 'relu')(inputs)
out = tf.keras.layers.Dense(layers[2], activation = 'relu')(out)
out = tf.keras.layers.Dense(layers[3])(out)

model = tf.keras.Model(inputs, out)

In [12]:
model_path = "../../models/"
arch_folder = "input-dense-dense-dense/"
model_name = "196_24_14_10.h5"
model_in_path = model_path+arch_folder+model_name

model.load_weights(model_in_path)
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 196)]             0         
                                                                 
 dense (Dense)               (None, 24)                4728      
                                                                 
 dense_1 (Dense)             (None, 14)                350       
                                                                 
 dense_2 (Dense)             (None, 10)                150       
                                                                 
Total params: 5228 (20.42 KB)
Trainable params: 5228 (20.42 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Generate Input Json

In [16]:
weights, biases = transfer_weights(layers, model, 36)

In [17]:
# Generate sample input
X = test_images_tf_downsampled[0:1]
#X = np.random.rand(1,layers[0])


X_in = [int(x*1e36) for x in X[0]]

In [18]:
x_in, dense_weights, dense_biases, dense_outs, dense_remainders, relu_outs, pred = prepare_input_json(layers, weights, biases, X_in, scalar=36, relu=True)
#np.argmax(dense_outs[-1]), relu_outs

In [19]:
in_json = {
    "in": x_in,
    "Dense32weights": dense_weights[0],
    "Dense32bias": dense_biases[0],
    "Dense32out": dense_outs[0],
    "Dense32remainder": dense_remainders[0],
    "ReLUout": relu_outs[0], 
    "Dense21weights": dense_weights[1],
    "Dense21bias": dense_biases[1],
    "Dense21out": dense_outs[1],
    "Dense21remainder": dense_remainders[1],
    "ReLUout2": relu_outs[1],
    "Dense10weights": dense_weights[2],
    "Dense10bias": dense_biases[2],
    "Dense10out": dense_outs[2],
    "Dense10remainder": dense_remainders[2]
}


with open("./tmp/input.json", "w") as f:
    json.dump(in_json, f)

In [54]:
model.predict(X), np.argmax(model.predict(X)), pred

1/1 [==============================] - 0s 13ms/step


(array([[ -2.7833426 ,  -5.505134  ,  -2.8890784 ,   0.39722544,
         -11.645189  ,  -6.587846  , -19.26069   ,   6.147421  ,
          -6.6978073 ,  -1.7802119 ]], dtype=float32),
 7,
 7)

### Benchmarking Proof Gen

In [22]:
zkey_1 = ceremony_folder + 'test_0000.zkey'
veri_key = ceremony_folder + 'vk.json'

In [23]:
test_size = 2500
loss, mem_usage, time_cost = benchmark_(test_images_tf_downsampled[:test_size], predictions_tf[:test_size], weights, biases, layers, "./tmp/", "./tmp/input.json", zkey_1, veri_key)

Loss happens on index 66
Loss happens on index 300
Loss happens on index 320
Loss happens on index 403
Loss happens on index 448
Loss happens on index 495
Loss happens on index 572
Loss happens on index 939
Loss happens on index 1247
Loss happens on index 1256
Loss happens on index 1272
Loss happens on index 1319
Loss happens on index 1465
Loss happens on index 1525
Loss happens on index 1562
Loss happens on index 1634
Loss happens on index 1641
Loss happens on index 2035
Loss happens on index 2129
Loss happens on index 2186
Loss happens on index 2298
Loss happens on index 2395
Total time: 1133.4991629123688


In [24]:
csv_path = '../../benchmarks/benchmark_results.csv'

columns = ['Framework', 'Architecture', '# Layers', '# Parameters', 'Testing Size', 'Accuracy Loss (%)', 
           'Avg Memory Usage (MB)', 'Std Memory Usage', 'Avg Proving Time (s)', 'Std Proving Time']

# Check if the CSV file exists
if not os.path.isfile(csv_path):
    # Create a DataFrame with the specified columns
    df = pd.DataFrame(columns=columns)
    # Save the DataFrame as a CSV file
    df.to_csv(csv_path, index=False)
else:
    print(f"File '{csv_path}' already exists.")

df = pd.read_csv(csv_path)

File '../../benchmarks/benchmark_results.csv' already exists.


In [25]:
new_row = {
    'Framework': ['circomlib-ml (tensorflow)'],
    'Architecture': ['Input-Dense-Dense (196x24x14x10)'],
    '# Layers': [4],
    '# Parameters': [5228],
    'Testing Size': [test_size],
    'Accuracy Loss (%)': [loss/test_size * 100],
    'Avg Memory Usage (MB)': [sum(mem_usage) / len(mem_usage)],
    'Std Memory Usage': [pd.Series(mem_usage).std()],
    'Avg Proving Time (s)': [sum(time_cost) / len(time_cost)],
    'Std Proving Time': [pd.Series(time_cost).std()]
}

new_row_df = pd.DataFrame(new_row)


In [26]:
df = pd.concat([df, new_row_df], ignore_index=True)
df.to_csv(csv_path, index=False)
df

,Framework,Architecture,# Layers,# Parameters,Testing Size,Accuracy Loss (%),Avg Memory Usage (MB),Std Memory Usage,Avg Proving Time (s),Std Proving Time
0,opml (pytorch),Input-Dense-Dense (196x25x10),3,5185,5000,3.52,74.351948,1.419725,0.809440,0.074357
1,opml (pytorch),Input-Dense-Dense (784x56x10),3,44543,2500,0.72,89.120883,2.254392,3.609974,0.421732
2,opml (pytorch),Input-Dense-Dense-Dense (196x24x14x10),4,5228,5000,3.42,74.450789,1.491530,0.829767,0.085084
3,circomlib-ml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,2500,0.00,998.735128,8.040170,1.169482,0.108491
4,circomlib-ml (tensorflow),Input-Dense-Dense (784x56x10,3,44543,2500,0.04,2328.322411,34.436486,2.351284,0.147113
5,zkml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,1000,0.10,2334.295629,12.335301,21.263430,0.294183
6,zkml (tensorflow),Input-Dense-Dense (784x56x10),3,44543,1000,0.00,2357.363156,11.750076,22.012543,0.317096
7,zkml (tensorflow),Input-Dense-Dense (196x24x14x10),4,5228,2500,0.00,2338.480847,13.611456,22.168541,0.528263
8,ezkl (pytorch),Input-Dense-Dense (196x25x10),3,5185,10000,4.11,91.608069,0.756873,0.660479,0.025701
9,ezkl (pytorch),Input-Dense-Dense-Dense (196x24x14x10),4,5228,10000,15.71,91.589901,0.742728,0.693572,0.024197


# CNN

## Prepare

In [33]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617

In [8]:
import numpy as np
import torch, json

# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images_tf = train_images / 255.0
test_images_tf = test_images / 255.0
train_images_tf = train_images_tf.reshape(train_images.shape[0], 28, 28, 1)
test_images_tf = test_images_tf.reshape(test_images.shape[0], 28, 28, 1)

train_images_tf_14 = tf.image.resize(train_images_tf, [14, 14]).numpy()
test_images_tf_14 = tf.image.resize(test_images_tf, [14, 14]).numpy()

In [3]:
# Convert to PyTorch format [batch_size, channels, height, width]
train_images_pt = torch.tensor(train_images_tf).permute(0, 3, 1, 2).float()
test_images_pt = torch.tensor(test_images_tf).permute(0, 3, 1, 2).float()


train_images_pt_14 =  torch.tensor(test_images_tf_14).permute(0, 3, 1, 2).float()
test_images_pt_14 =  torch.tensor(test_images_tf_14).permute(0, 3, 1, 2).float()

In [4]:
def Conv2DInt(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    Input = [[[str(input[i][j][k] % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    Weights = [[[[str(weights[i][j][k][l] % p) for l in range(nFilters)] for k in range(nChannels)] for j in range(kernelSize)] for i in range(kernelSize)]
    Bias = [str(bias[i] % p) for i in range(nFilters)]
    out = [[[0 for _ in range(nFilters)] for _ in range((nCols - kernelSize)//strides + 1)] for _ in range((nRows - kernelSize)//strides + 1)]
    remainder = [[[None for _ in range(nFilters)] for _ in range((nCols - kernelSize)//strides + 1)] for _ in range((nRows - kernelSize)//strides + 1)]
    for i in range((nRows - kernelSize)//strides + 1):
        for j in range((nCols - kernelSize)//strides + 1):
            for m in range(nFilters):
                for k in range(nChannels):
                    for x in range(kernelSize):
                        for y in range(kernelSize):
                            out[i][j][m] += input[i*strides+x][j*strides+y][k] * weights[x][y][k][m]
                out[i][j][m] += bias[m]
                remainder[i][j][m] = str(out[i][j][m] % n)
                out[i][j][m] = str(out[i][j][m] // n % p)
    return Input, Weights, Bias, out, remainder

In [5]:
def AveragePooling2DInt (nRows, nCols, nChannels, poolSize, strides, input):
    Input = [[[str(input[i][j][k] % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    out = [[[0 for _ in range(nChannels)] for _ in range((nCols-poolSize)//strides + 1)] for _ in range((nRows-poolSize)//strides + 1)]
    remainder = [[[None for _ in range(nChannels)] for _ in range((nCols-poolSize)//strides + 1)] for _ in range((nRows-poolSize)//strides + 1)]
    for i in range((nRows-poolSize)//strides + 1):
        for j in range((nCols-poolSize)//strides + 1):
            for k in range(nChannels):
                for x in range(poolSize):
                    for y in range(poolSize):
                        out[i][j][k] += input[i*strides+x][j*strides+y][k]
                remainder[i][j][k] = str(out[i][j][k] % poolSize**2 % p)
                out[i][j][k] = str(out[i][j][k] // poolSize**2 % p)
    return Input, out, remainder

In [6]:
def DenseInt(nInputs, nOutputs, n, input, weights, bias):
    Input = [str(input[i] % p) for i in range(nInputs)]
    Weights = [[str(weights[i][j] % p) for j in range(nOutputs)] for i in range(nInputs)]
    Bias = [str(bias[i] % p) for i in range(nOutputs)]
    out = [0 for _ in range(nOutputs)]
    remainder = [None for _ in range(nOutputs)]
    for j in range(nOutputs):
        for i in range(nInputs):
            out[j] += input[i] * weights[i][j]
        out[j] += bias[j]
        remainder[j] = str(out[j] % n)
        out[j] = str(out[j] // n % p)
    return Input, Weights, Bias, out, remainder

In [9]:
def prepare_input_json_cnn(X, layers, input_path):
    kernal_size = layers[-1]
    X_in = [[[int(X[i][j][0]*1e18)] for j in range(layers[0])] for i in range(layers[0])]
    conv2d_1_weights = [[[[int(model.layers[1].weights[0][i][j][k][l]*1e18) for l in range(layers[1])] for k in range(1)] for j in range(kernal_size)] for i in range(kernal_size)]
    conv2d_1_bias = [int(model.layers[1].weights[1][i]*1e36) for i in range(layers[1])]

    X_in, conv2d_1_weights, conv2d_1_bias, conv2d_1_out, conv2d_1_remainder = Conv2DInt(layers[0], layers[0], 1, layers[1], kernal_size, 1, 10**18, X_in, conv2d_1_weights, conv2d_1_bias)

    relu_1_in = [[[int(conv2d_1_out[i][j][k]) for k in range(5)] for j in range(12)] for i in range(12)]
    relu_1_out = [[[str(relu_1_in[i][j][k]) if relu_1_in[i][j][k] < p//2 else 0 for k in range(5)] for j in range(12)] for i in range(12)]

    avg2d_1_in = [[[int(relu_1_out[i][j][k]) for k in range(5)] for j in range(12)] for i in range(12)]

    _, avg2d_1_out, avg2d_1_remainder = AveragePooling2DInt(12, 12, 5, 2, 2, avg2d_1_in)


    conv2d_2_in = [[[int(avg2d_1_out[i][j][k]) for k in range(5)] for j in range(6)] for i in range(6)]
    conv2d_2_weights = [[[[int(model.layers[4].weights[0][i][j][k][l]*1e18) for l in range(11)] for k in range(5)] for j in range(kernal_size)] for i in range(kernal_size)]
    conv2d_2_bias = [int(model.layers[4].weights[1][i]*1e36) for i in range(11)]

    _, conv2d_2_weights, conv2d_2_bias, conv2d_2_out, conv2d_2_remainder = Conv2DInt(6, 6, 5, 11, 3, 1, 10**18, conv2d_2_in, conv2d_2_weights, conv2d_2_bias)


    relu_2_in = [[[int(conv2d_2_out[i][j][k]) for k in range(11)] for j in range(4)] for i in range(4)]
    relu_2_out = [[[str(relu_2_in[i][j][k]) if relu_2_in[i][j][k] < p//2 else 0 for k in range(11)] for j in range(4)] for i in range(4)]


    avg2d_2_in = [[[int(relu_2_out[i][j][k]) if int(relu_2_out[i][j][k]) < p//2 else int(relu_2_out[i][j][k]) - p for k in range(11)] for j in range(4)] for i in range(4)]

    _, avg2d_2_out, avg2d_2_remainder = AveragePooling2DInt(4, 4, 11, 2, 2, avg2d_2_in)

    flatten_out = [avg2d_2_out[i][j][k] for i in range(2) for j in range(2) for k in range(11)]

    flatten_model = tf.keras.Model(inputs, model.layers[7].output)
    flatten_model.predict(X.reshape(1,14,14,1))[0][00:20]

    dense_in = [int(flatten_out[i]) if int(flatten_out[i]) < p//2 else int(flatten_out[i]) - p for i in range(44)]
    dense_weights = [[int(model.layers[8].weights[0][i][j]*1e18) for j in range(80)] for i in range(44)]
    dense_bias = [int(model.layers[8].weights[1][i]*1e36) for i in range(80)]

    _, dense_weights, dense_bias, dense_out, dense_remainder = DenseInt(44, 80, 10**18, dense_in, dense_weights, dense_bias)

    dense_1_weights = [[int(model.layers[10].weights[0][i][j]*1e18) for j in range(10)] for i in range(80)]
    dense_1_bias = [int(model.layers[10].weights[1][i]*1e36) for i in range(10)]

    re_lu_3_out = [dense_out[i] if int(dense_out[i]) < p//2 else 0 for i in range(80)]
    dense_1_in = [int(re_lu_3_out[i]) for i in range(80)]

    _, dense_1_weights, dense_1_bias, dense_1_out, dense_1_remainder = DenseInt(80, 10, 10**18, dense_1_in, dense_1_weights, dense_1_bias)

    in_json = {
        "in": X_in,
        "conv2d_2_weights": conv2d_1_weights,
        "conv2d_2_bias": conv2d_1_bias,
        "conv2d_2_out": conv2d_1_out,
        "conv2d_2_remainder": conv2d_1_remainder,
        "re_lu_1_out": relu_1_out,
        "average_pooling2d_1_out": avg2d_1_out,
        "average_pooling2d_1_remainder": avg2d_1_remainder,
        "conv2d_3_weights": conv2d_2_weights,
        "conv2d_3_bias": conv2d_2_bias,
        "conv2d_3_out": conv2d_2_out,
        "conv2d_3_remainder": conv2d_2_remainder,
        "re_lu_2_out": relu_2_out,
        "average_pooling2d_2_out": avg2d_2_out,
        "average_pooling2d_2_remainder": avg2d_2_remainder,
        "flatten_out": flatten_out,
        "dense_weights": dense_weights,
        "dense_bias": dense_bias,
        "dense_out": dense_out,
        "dense_remainder": dense_remainder,
        "re_lu_3_out":re_lu_3_out,
        "dense_1_weights":dense_1_weights,
        "dense_1_bias":dense_1_bias,
        "dense_1_out":dense_1_out,
        "dense_1_remainder":dense_1_remainder
    }

    out = [int(x) for x in dense_1_out]
    out = [x if x < p//2 else 0 for x in out]
    pred = np.argmax(out)

    with open(input_path, "w") as f:
        json.dump(in_json, f)

    return pred

In [29]:
def benchmark_cnn(test_images, predictions, layers, tmp_folder, input_path, zkey, veri_key):
    loss = 0

    target_circom = "_".join(str(x) for x in layers) + '.circom'

    json_folder = tmp_folder + target_circom[:-7] + "_js/"
    wit_json_file = json_folder + "generate_witness.js"
    wasm_file = json_folder + target_circom[:-7] + ".wasm"
    input_path = tmp_folder + "input.json"
    wit_file = tmp_folder + "witness.wtns"

    mem_usage = []
    time_cost = []
    benchmark_start_time = time.time()

    for i in range(len(test_images)):
        cost = 0
        X = test_images[i]
        start_time = time.time()
        pred = prepare_input_json_cnn(X, layers, input_path)

        if pred != predictions[i]:
            loss += 1
            print ("Loss happens on index", i)

        commands = [['node', wit_json_file, wasm_file, input_path, wit_file],
                    ['snarkjs', 'groth16', 'prove',zkey, wit_file, tmp_folder+'proof.json', tmp_folder+'public.json']]
                    #['snarkjs', 'groth16', 'verify',veri_key, tmp_folder+'public.json', tmp_folder+'proof.json']]

        for command in commands:
            _, _, usage = execute_and_monitor(command)
            cost += usage
        #print ("stdout:", stdout)
            
        mem_usage.append(cost)
        time_cost.append(time.time() - start_time)
    
    print ("Total time:", time.time() - benchmark_start_time)
    return loss, mem_usage, time_cost

## For 5_11_80_10 CNN for 14x14 input and 3x3 kernal size

In [10]:
layers = [14, 5, 11, 80, 10, 3]
kernal_size = layers[-1]

# Define the LeNet model in TensorFlow
inputs = tf.keras.layers.Input(shape=(layers[0],layers[0],1))
out = tf.keras.layers.Conv2D(layers[1],kernal_size, use_bias = True)(inputs)
out = tf.keras.layers.ReLU()(out)
out = tf.keras.layers.AveragePooling2D()(out)
out = tf.keras.layers.Conv2D(layers[2],kernal_size, use_bias = True)(out)
out = tf.keras.layers.ReLU()(out)
out = tf.keras.layers.AveragePooling2D()(out)
out = tf.keras.layers.Flatten()(out)
out = tf.keras.layers.Dense(layers[3])(out)
out = tf.keras.layers.ReLU()(out)
out = tf.keras.layers.Dense(layers[4])(out)
model = tf.keras.Model(inputs, out)

# Compile the model
model.compile(optimizer='adam',
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])


In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14, 14, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 12, 12, 5)         50        
                                                                 
 re_lu (ReLU)                (None, 12, 12, 5)         0         
                                                                 
 average_pooling2d (Average  (None, 6, 6, 5)           0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 11)          506       
                                                                 
 re_lu_1 (ReLU)              (None, 4, 4, 11)          0         
                                                             

In [12]:
model_in_path

NameError: name 'model_in_path' is not defined

In [13]:
model_path = "../../models/"
arch_folder = "input-conv2d-conv2d-dense-dense/"
model_name = "_".join([str(x) for x in layers]) +".h5"
model_in_path = model_path+arch_folder+model_name

model.load_weights(model_in_path)
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14, 14, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 12, 12, 5)         50        
                                                                 
 re_lu (ReLU)                (None, 12, 12, 5)         0         
                                                                 
 average_pooling2d (Average  (None, 6, 6, 5)           0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 11)          506       
                                                                 
 re_lu_1 (ReLU)              (None, 4, 4, 11)          0         
                                                             

In [14]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images_tf_14, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

2024-02-06 18:03:21.375630: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-06 18:03:21.435849: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-06 18:03:21.669856: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


313/313 - 1s - loss: 0.7188 - accuracy: 0.9707 - 1s/epoch - 4ms/step

Test accuracy: 0.9707000255584717


### Benchmark

In [21]:
predictions_tf = get_predictions_tf(model, test_images_tf_14)

1/1 [==============================] - 0s 44ms/step


In [24]:
tmp_folder = './tmp/'
ceremony_folder = tmp_folder + 'very_big_ceremony/'

In [25]:
zkey_1 = ceremony_folder + 'test_0000.zkey'
veri_key = ceremony_folder + 'vk.json'

In [27]:
layers

[14, 5, 11, 80, 10, 3]

In [43]:
test_size = 250
loss, mem_usage, time_cost = benchmark_cnn(test_images_tf_14[:test_size], predictions_tf[:test_size], layers, "./tmp/", "./tmp/input.json", zkey_1, veri_key)

1/1 [==============================] - 0s 39ms/step
Total time: 1903.5131404399872


In [44]:
csv_path = '../../benchmarks/benchmark_results.csv'

columns = ['Framework', 'Architecture', '# Layers', '# Parameters', 'Testing Size', 'Accuracy Loss (%)', 
           'Avg Memory Usage (MB)', 'Std Memory Usage', 'Avg Proving Time (s)', 'Std Proving Time']

# Check if the CSV file exists
if not os.path.isfile(csv_path):
    # Create a DataFrame with the specified columns
    df = pd.DataFrame(columns=columns)
    # Save the DataFrame as a CSV file
    df.to_csv(csv_path, index=False)
else:
    print(f"File '{csv_path}' already exists.")

df = pd.read_csv(csv_path)

File '../../benchmarks/benchmark_results.csv' already exists.


In [45]:
new_row = {
    'Framework': ['circomlib-ml (tensorflow)'],
    'Architecture': ['Input-Conv2d-Conv2d-Dense-Dense (196x5x11x80x10)'],
    '# Layers': [5],
    '# Parameters': [4996],
    'Testing Size': [test_size],
    'Accuracy Loss (%)': [loss/test_size * 100],
    'Avg Memory Usage (MB)': [sum(mem_usage) / len(mem_usage)],
    'Std Memory Usage': [pd.Series(mem_usage).std()],
    'Avg Proving Time (s)': [sum(time_cost) / len(time_cost)],
    'Std Proving Time': [pd.Series(time_cost).std()]
}

new_row_df = pd.DataFrame(new_row)

In [46]:
df = pd.concat([df, new_row_df], ignore_index=True)
df.to_csv(csv_path, index=False)
df

,Framework,Architecture,# Layers,# Parameters,Testing Size,Accuracy Loss (%),Avg Memory Usage (MB),Std Memory Usage,Avg Proving Time (s),Std Proving Time,Notes,Notes
0,opml (pytorch),Input-Dense-Dense (196x25x10),3,5185,5000.0,3.52,74.3519484375,1.419725,0.809440,0.074357,NaN,NaN
1,opml (pytorch),Input-Dense-Dense-Dense (196x24x14x10),4,5228,5000.0,3.42,74.4507890625,1.491530,0.829767,0.085084,NaN,NaN
2,opml (pytorch),Input-Dense-Dense (784x56x10),3,44543,2500.0,0.72,89.1208828125,2.254392,3.609974,0.421732,NaN,NaN
3,circomlib-ml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,2500.0,0.00,998.735128125,8.040170,1.169482,0.108491,non-linear constraints: 18075,NaN
4,circomlib-ml (tensorflow),Input-Dense-Dense-Dense (196x24x14x10),4,5228,2500.0,0.88,315.435559375,1.690681,0.452942,0.031613,non-linear constraints: 24826,NaN
5,circomlib-ml (tensorflow),Input-Dense-Dense (784x56x10),3,44543,2500.0,0.04,2328.3224109375,34.436486,2.351284,0.147113,non-linear constraints: 73416,NaN
6,circomlib-ml (tensorflow),Input-Conv2d-Conv2d-Dense-Dense (196x5x11x80x10),5,4996,100.0,0.00,4276.0376171875,157.095080,9.522366,1.024470,NaN,NaN
7,circomlib-ml (tensorflow),Input-Conv2d-Conv2d-Dense (784x6x16x10),4,5142,NaN,NaN,.,NaN,NaN,NaN,non-linear constraints: 2558720,NaN
8,zkml (tensorflow),Input-Dense-Dense (196x25x10),3,5185,1000.0,0.10,2334.29562890625,12.335301,21.263430,0.294183,NaN,NaN
9,zkml (tensorflow),Input-Dense-Dense-Dense (196x24x14x10),4,5228,2500.0,0.00,2338.480846875,13.611456,22.168541,0.528263,NaN,NaN


### Test

In [28]:
X = test_images_tf_14[0]
X_in = [[[int(X[i][j][0]*1e18)] for j in range(layers[0])] for i in range(layers[0])]

In [115]:
conv2d_1_weights = [[[[int(model.layers[1].weights[0][i][j][k][l]*1e18) for l in range(layers[1])] for k in range(1)] for j in range(kernal_size)] for i in range(kernal_size)]
conv2d_1_bias = [int(model.layers[1].weights[1][i]*1e36) for i in range(layers[1])]

In [116]:
X_in, conv2d_1_weights, conv2d_1_bias, conv2d_1_out, conv2d_1_remainder = Conv2DInt(layers[0], layers[0], 1, layers[1], kernal_size, 1, 10**18, X_in, conv2d_1_weights, conv2d_1_bias)
conv2d_1_out[0][0]


['77728135713381770',
 '39929983748117169',
 '68350061524653486',
 '73588518691373187',
 '21888242871839275222246405745257275088548364400416034343698107745471066227552']

In [117]:
conv2d_model = tf.keras.Model(inputs, model.layers[1].output)
conv2d_model.predict(X.reshape(1,14,14,1))[0][0][0]

1/1 [==============================] - 0s 17ms/step


array([ 0.07772814,  0.03992999,  0.06835006,  0.07358852, -0.09644111],
      dtype=float32)

In [118]:
relu_1_in = [[[int(conv2d_1_out[i][j][k]) for k in range(5)] for j in range(12)] for i in range(12)]
relu_1_out = [[[str(relu_1_in[i][j][k]) if relu_1_in[i][j][k] < p//2 else 0 for k in range(5)] for j in range(12)] for i in range(12)]

In [119]:
avg2d_1_in = [[[int(relu_1_out[i][j][k]) for k in range(5)] for j in range(12)] for i in range(12)]


In [120]:
_, avg2d_1_out, avg2d_1_remainder = AveragePooling2DInt(12, 12, 5, 2, 2, avg2d_1_in)
avg2d_1_out[0][5]


['77728135713381770',
 '39929983748117169',
 '68350061524653486',
 '73588518691373187',
 '0']

In [121]:
avg2d_1_model = tf.keras.Model(inputs, model.layers[2].output)
avg2d_1_model.predict(X.reshape(1,14,14,1))[0][0][5]

1/1 [==============================] - 0s 17ms/step


array([0.07772814, 0.03992999, 0.06835006, 0.07358852, 0.        ],
      dtype=float32)

In [122]:
conv2d_2_in = [[[int(avg2d_1_out[i][j][k]) for k in range(5)] for j in range(6)] for i in range(6)]
conv2d_2_weights = [[[[int(model.layers[4].weights[0][i][j][k][l]*1e18) for l in range(11)] for k in range(5)] for j in range(kernal_size)] for i in range(kernal_size)]
conv2d_2_bias = [int(model.layers[4].weights[1][i]*1e36) for i in range(11)]

In [123]:
_, conv2d_2_weights, conv2d_2_bias, conv2d_2_out, conv2d_2_remainder = Conv2DInt(6, 6, 5, 11, 3, 1, 10**18, conv2d_2_in, conv2d_2_weights, conv2d_2_bias)
conv2d_2_out[0][0]


['21888242871839275222246405745257275088548364400416034343697375192694143195731',
 '1061651740198247588',
 '764814615205212968',
 '2001471352560688122',
 '478249238255680309',
 '21888242871839275222246405745257275088548364400416034343698135246681197718355',
 '171914590268020896',
 '2053477019572830122',
 '1557245406039370781',
 '21888242871839275222246405745257275088548364400416034343697919028904502137351',
 '1179194581603657116']

In [124]:
conv2d_2_model = tf.keras.Model(inputs, model.layers[4].output)
conv2d_2_model.predict(X.reshape(1,14,14,1))[0][0][0]

1/1 [==============================] - 0s 21ms/step


array([-0.8289941 ,  1.0616518 ,  0.76481473,  2.0014715 ,  0.47824925,
       -0.06893994,  0.17191476,  2.053477  ,  1.5572454 , -0.28515762,
        1.1791945 ], dtype=float32)

In [125]:
relu_2_in = [[[int(conv2d_2_out[i][j][k]) for k in range(11)] for j in range(4)] for i in range(4)]
relu_2_out = [[[str(relu_2_in[i][j][k]) if relu_2_in[i][j][k] < p//2 else 0 for k in range(11)] for j in range(4)] for i in range(4)]
relu_2_out[0][0]


[0,
 '1061651740198247588',
 '764814615205212968',
 '2001471352560688122',
 '478249238255680309',
 0,
 '171914590268020896',
 '2053477019572830122',
 '1557245406039370781',
 0,
 '1179194581603657116']

In [126]:
relu_2_model = tf.keras.Model(inputs, model.layers[5].output)
relu_2_model.predict(X.reshape(1,14,14,1))[0][0][0]

1/1 [==============================] - 0s 20ms/step


array([0.        , 1.0616518 , 0.76481473, 2.0014715 , 0.47824925,
       0.        , 0.17191476, 2.053477  , 1.5572454 , 0.        ,
       1.1791945 ], dtype=float32)

In [127]:
avg2d_2_in = [[[int(relu_2_out[i][j][k]) if int(relu_2_out[i][j][k]) < p//2 else int(relu_2_out[i][j][k]) - p for k in range(11)] for j in range(4)] for i in range(4)]


In [128]:
_, avg2d_2_out, avg2d_2_remainder = AveragePooling2DInt(4, 4, 11, 2, 2, avg2d_2_in)
avg2d_2_out[1][1]

['2102647666634922773',
 '894141372900805973',
 '1144439275149027474',
 '363196598361861398',
 '213998246616241930',
 '2252287174178330193',
 '445970532392696869',
 '0',
 '803604884783306802',
 '714274412066657307',
 '569926069615134546']

In [129]:
avg2d_2_model = tf.keras.Model(inputs, model.layers[6].output)
avg2d_2_model.predict(X.reshape(1,14,14,1))[0][1][1]

1/1 [==============================] - 0s 22ms/step


array([2.1026475 , 0.8941413 , 1.1444392 , 0.3631966 , 0.21399826,
       2.2522874 , 0.44597048, 0.        , 0.80360484, 0.71427447,
       0.569926  ], dtype=float32)

In [130]:
flatten_out = [avg2d_2_out[i][j][k] for i in range(2) for j in range(2) for k in range(11)]
flatten_out[00:20]

['477108045675720547',
 '454348809073881608',
 '1222294054130137075',
 '1727249025601025379',
 '644318262656794661',
 '166953333726138123',
 '205594658904789491',
 '1995762449806793120',
 '829734313059824263',
 '945030729295821005',
 '2693527205682013079',
 '479539956960218692',
 '563633363111811675',
 '152797929822735732',
 '1833163960437009507',
 '1921918850361998422',
 '1031197185005236028',
 '485808131355185802',
 '631620443451063835',
 '1564688588168701320']

In [131]:
flatten_model = tf.keras.Model(inputs, model.layers[7].output)
flatten_model.predict(X.reshape(1,14,14,1))[0][00:20]

1/1 [==============================] - 0s 23ms/step


array([0.4771081 , 0.45434898, 1.2222941 , 1.7272491 , 0.6443184 ,
       0.16695338, 0.20559475, 1.9957625 , 0.8297342 , 0.9450308 ,
       2.6935272 , 0.47954   , 0.5636334 , 0.15279782, 1.8331639 ,
       1.9219189 , 1.0311973 , 0.4858083 , 0.6316203 , 1.5646884 ],
      dtype=float32)

In [132]:
dense_in = [int(flatten_out[i]) if int(flatten_out[i]) < p//2 else int(flatten_out[i]) - p for i in range(44)]
dense_weights = [[int(model.layers[8].weights[0][i][j]*1e18) for j in range(80)] for i in range(44)]
dense_bias = [int(model.layers[8].weights[1][i]*1e36) for i in range(80)]

In [133]:
_, dense_weights, dense_bias, dense_out, dense_remainder = DenseInt(44, 80, 10**18, dense_in, dense_weights, dense_bias)
dense_out

['21888242871839275222246405745257275088548364400416034343696761842086101753305',
 '21888242871839275222246405745257275088548364400416034343697150589723119874502',
 '21888242871839275222246405745257275088548364400416034343696866998423020810756',
 '1865943277987052021',
 '21888242871839275222246405745257275088548364400416034343696594162432229542914',
 '327671754658778462',
 '21888242871839275222246405745257275088548364400416034343698003775064915090839',
 '21888242871839275222246405745257275088548364400416034343697791656096357106527',
 '21888242871839275222246405745257275088548364400416034343697037474389099867663',
 '1469284766102098672',
 '2360525966081655866',
 '2257995715134204875',
 '21888242871839275222246405745257275088548364400416034343698168893781783189437',
 '709033679885699524',
 '1053898167324235066',
 '21888242871839275222246405745257275088548364400416034343696008370121249529633',
 '21888242871839275222246405745257275088548364400416034343696567084602867134671',
 '265498158176

In [134]:
dense_1_weights = [[int(model.layers[10].weights[0][i][j]*1e18) for j in range(10)] for i in range(80)]
dense_1_bias = [int(model.layers[10].weights[1][i]*1e36) for i in range(10)]

In [135]:
re_lu_3_out = [dense_out[i] if int(dense_out[i]) < p//2 else 0 for i in range(80)]
dense_1_in = [int(re_lu_3_out[i]) for i in range(80)]


In [136]:
_, dense_1_weights, dense_1_bias, dense_1_out, dense_1_remainder = DenseInt(80, 10, 10**18, dense_1_in, dense_1_weights, dense_1_bias)
dense_1_out



['21888242871839275222246405745257275088548364400416034343692983529519237817330',
 '21888242871839275222246405745257275088548364400416034343690991325412587692936',
 '485787271832041756',
 '21888242871839275222246405745257275088548364400416034343697221273515064929337',
 '21888242871839275222246405745257275088548364400416034343688734116644544946776',
 '21888242871839275222246405745257275088548364400416034343692742618111832500426',
 '21888242871839275222246405745257275088548364400416034343675710831992639808693',
 '8961098346234734865',
 '21888242871839275222246405745257275088548364400416034343691493709225102018279',
 '851121769057298969']

In [137]:
dense_model = tf.keras.Model(inputs, model.layers[-1].output)
dense_model.predict(X.reshape(1,14,14,1))[0]



1/1 [==============================] - 0s 45ms/step


array([ -5.220658  ,  -7.212863  ,   0.485788  ,  -0.9829123 ,
        -9.470071  ,  -5.4615684 , -22.493355  ,   8.961097  ,
        -6.710476  ,   0.85112095], dtype=float32)

In [143]:
import json, subprocess
in_json = {
    "in": X_in,
    "conv2d_2_weights": conv2d_1_weights,
    "conv2d_2_bias": conv2d_1_bias,
    "conv2d_2_out": conv2d_1_out,
    "conv2d_2_remainder": conv2d_1_remainder,
    "re_lu_1_out": relu_1_out,
    "average_pooling2d_1_out": avg2d_1_out,
    "average_pooling2d_1_remainder": avg2d_1_remainder,
    "conv2d_3_weights": conv2d_2_weights,
    "conv2d_3_bias": conv2d_2_bias,
    "conv2d_3_out": conv2d_2_out,
    "conv2d_3_remainder": conv2d_2_remainder,
    "re_lu_2_out": relu_2_out,
    "average_pooling2d_2_out": avg2d_2_out,
    "average_pooling2d_2_remainder": avg2d_2_remainder,
    "flatten_out": flatten_out,
    "dense_weights": dense_weights,
    "dense_bias": dense_bias,
    "dense_out": dense_out,
    "dense_remainder": dense_remainder,
    "re_lu_3_out":re_lu_3_out,
    "dense_1_weights":dense_1_weights,
    "dense_1_bias":dense_1_bias,
    "dense_1_out":dense_1_out,
    "dense_1_remainder":dense_1_remainder
}

In [144]:
with open("input.json", "w") as f:
    json.dump(in_json, f)

In [141]:
tmp_folder = "./tmp/"
circuit_folder = "./golden_circuits/"
target_circom = "_".join(str(x) for x in layers) + '.circom'

command = ['circom', circuit_folder + target_circom, "--r1cs", "--wasm", "--sym", "-o", tmp_folder]
subprocess.run(command)

template instances: 27
non-linear constraints: 523312
linear constraints: 0
public inputs: 0
private inputs: 8602 (7412 belong to witness)
public outputs: 10
wires: 527717
labels: 1136943
Written successfully: ./tmp/14_5_11_80_10_3.r1cs
Written successfully: ./tmp/14_5_11_80_10_3.sym
Written successfully: ./tmp/14_5_11_80_10_3_js/14_5_11_80_10_3.wasm
Everything went okay


CompletedProcess(args=['circom', './golden_circuits/14_5_11_80_10_3.circom', '--r1cs', '--wasm', '--sym', '-o', './tmp/'], returncode=0)

In [145]:
# Gen Witness
json_folder = tmp_folder + target_circom[:-7] + "_js/"
wit_json_file = json_folder + "generate_witness.js"
wasm_file = json_folder + target_circom[:-7] + ".wasm"
input_path = "input.json"
wit_file = tmp_folder + "witness.wtns"

command = ['node', wit_json_file, wasm_file, input_path, wit_file]
subprocess.run(command)

CompletedProcess(args=['node', './tmp/14_5_11_80_10_3_js/generate_witness.js', './tmp/14_5_11_80_10_3_js/14_5_11_80_10_3.wasm', 'input.json', './tmp/witness.wtns'], returncode=0)

In [149]:
ceremony_folder = "./tmp/very_big_ceremony/"
ptau_2 = ceremony_folder + 'pot12_0001.ptau'
ptau_3 = ceremony_folder + 'pot12_final.ptau'

In [150]:

r1cs_path = tmp_folder + "_".join(str(x) for x in layers) + ".r1cs"
zkey_1 = ceremony_folder + 'test_0000.zkey'
command = ['snarkjs', 'groth16', 'setup', r1cs_path, ptau_3, zkey_1]
subprocess.run(command)

[INFO]  snarkJS: Reading r1cs
[INFO]  snarkJS: Reading tauG1
[INFO]  snarkJS: Reading tauG2
[INFO]  snarkJS: Reading alphatauG1
[INFO]  snarkJS: Reading betatauG1
[INFO]  snarkJS: Circuit hash: 
		d7afe5de 673733b7 83192f42 9cb8fb0f
		b78413bb 0f36716d 8620df82 3d2dc5c2
		80f2c9d8 38a5a5a7 ab5b755b 1bc56636
		ab8154e6 b6b81abf 6b774a72 7951bbb9


CompletedProcess(args=['snarkjs', 'groth16', 'setup', './tmp/14_5_11_80_10_3.r1cs', './tmp/very_big_ceremony/pot12_final.ptau', './tmp/very_big_ceremony/test_0000.zkey'], returncode=0)

In [151]:
command = ['snarkjs', 'groth16', 'prove',zkey_1, wit_file, tmp_folder+'proof.json', tmp_folder+'public.json']
subprocess.run(command)

CompletedProcess(args=['snarkjs', 'groth16', 'prove', './tmp/very_big_ceremony/test_0000.zkey', './tmp/witness.wtns', './tmp/proof.json', './tmp/public.json'], returncode=0)

## For 784_6_16_10 CNN

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [194]:
layers = [28, 6, 16, 10]
kernal_size = 5

inputs = tf.keras.layers.Input(shape=(layers[0],layers[0],1))
out = tf.keras.layers.Conv2D(layers[1],kernal_size, use_bias = True)(inputs)
out = tf.keras.layers.ReLU()(out)
out = tf.keras.layers.AveragePooling2D()(out)
out = tf.keras.layers.Conv2D(layers[2],kernal_size, use_bias = True)(out)
out = tf.keras.layers.ReLU()(out)
out = tf.keras.layers.AveragePooling2D()(out)
out = tf.keras.layers.Flatten()(out)
out = tf.keras.layers.Dense(layers[3])(out)

model = tf.keras.Model(inputs, out)
model.summary()

Model: "model_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_59 (Conv2D)          (None, 24, 24, 6)         156       
                                                                 
 re_lu_38 (ReLU)             (None, 24, 24, 6)         0         
                                                                 
 average_pooling2d_58 (Aver  (None, 12, 12, 6)         0         
 agePooling2D)                                                   
                                                                 
 conv2d_60 (Conv2D)          (None, 8, 8, 16)          2416      
                                                                 
 re_lu_39 (ReLU)             (None, 8, 8, 16)          0         
                                                          

In [195]:
model_path = "../../models/"
arch_folder = "input-conv2d-conv2d-dense/"
model_name = "784_6_16_10.h5"
model_in_path = model_path+arch_folder+model_name

model.load_weights(model_in_path)
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

Model: "model_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_59 (Conv2D)          (None, 24, 24, 6)         156       
                                                                 
 re_lu_38 (ReLU)             (None, 24, 24, 6)         0         
                                                                 
 average_pooling2d_58 (Aver  (None, 12, 12, 6)         0         
 agePooling2D)                                                   
                                                                 
 conv2d_60 (Conv2D)          (None, 8, 8, 16)          2416      
                                                                 
 re_lu_39 (ReLU)             (None, 8, 8, 16)          0         
                                                          

In [196]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 0s - loss: 0.3388 - accuracy: 0.9866 - 390ms/epoch - 1ms/step

Test accuracy: 0.9865999817848206


In [199]:
model.predict(X.reshape(1,28,28,1)).argmax()

1/1 [==============================] - 0s 26ms/step


7

In [175]:
def Conv2DInt(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    Input = [[[str(input[i][j][k] % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    Weights = [[[[str(weights[i][j][k][l] % p) for l in range(nFilters)] for k in range(nChannels)] for j in range(kernelSize)] for i in range(kernelSize)]
    Bias = [str(bias[i] % p) for i in range(nFilters)]
    out = [[[0 for _ in range(nFilters)] for _ in range((nCols - kernelSize)//strides + 1)] for _ in range((nRows - kernelSize)//strides + 1)]
    remainder = [[[None for _ in range(nFilters)] for _ in range((nCols - kernelSize)//strides + 1)] for _ in range((nRows - kernelSize)//strides + 1)]
    for i in range((nRows - kernelSize)//strides + 1):
        for j in range((nCols - kernelSize)//strides + 1):
            for m in range(nFilters):
                for k in range(nChannels):
                    for x in range(kernelSize):
                        for y in range(kernelSize):
                            out[i][j][m] += input[i*strides+x][j*strides+y][k] * weights[x][y][k][m]
                out[i][j][m] += bias[m]
                remainder[i][j][m] = str(out[i][j][m] % n)
                out[i][j][m] = str(out[i][j][m] // n % p)
    return Input, Weights, Bias, out, remainder

### Test

In [200]:
X = test_images[0]
X_in = [[[int(X[i][j][0]*1e18)] for j in range(layers[0])] for i in range(layers[0])]

In [201]:
#X_in = [[[int(X[i][j][0]*1e18)] for j in range(layers[0])] for i in range(layers[0])]
conv2d_1_weights = [[[[int(model.layers[1].weights[0][i][j][k][l]*1e18) for l in range(layers[1])] for k in range(1)] for j in range(kernal_size)] for i in range(kernal_size)]
conv2d_1_bias = [int(model.layers[1].weights[1][i]*1e36) for i in range(layers[1])]

In [202]:
X_in, conv2d_1_weights, conv2d_1_bias, conv2d_1_out, conv2d_1_remainder = Conv2DInt(layers[0], layers[0], 1, layers[1], kernal_size, 1, 10**18, X_in, conv2d_1_weights, conv2d_1_bias)
conv2d_1_out[0][0]


['21888242871839275222246405745257275088548364400416034343698092813388794765723',
 '21888242871839275222246405745257275088548364400416034343698115293231099831772',
 '21888242871839275222246405745257275088548364400416034343698188535797543706346',
 '18634872343567936',
 '21888242871839275222246405745257275088548364400416034343698199850860109198944',
 '21888242871839275222246405745257275088548364400416034343698189564233562503226']

In [203]:
conv2d_model = tf.keras.Model(inputs, model.layers[1].output)
conv2d_model.predict(X.reshape(1,28,28,1))[0][0][0]

1/1 [==============================] - 0s 28ms/step


array([-0.11137319, -0.08889335, -0.01565078,  0.01863487, -0.00433572,
       -0.01462234], dtype=float32)

In [204]:
np.array(conv2d_1_out).shape

(24, 24, 6)

In [205]:
relu_1_in = [[[int(conv2d_1_out[i][j][k]) for k in range(6)] for j in range(24)] for i in range(24)]
relu_1_out = [[[str(relu_1_in[i][j][k]) if relu_1_in[i][j][k] < p//2 else 0 for k in range(6)] for j in range(24)] for i in range(24)]

In [206]:
avg2d_1_in = [[[int(relu_1_out[i][j][k]) for k in range(6)] for j in range(24)] for i in range(24)]


In [207]:
def AveragePooling2DInt (nRows, nCols, nChannels, poolSize, strides, input):
    Input = [[[str(input[i][j][k] % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    out = [[[0 for _ in range(nChannels)] for _ in range((nCols-poolSize)//strides + 1)] for _ in range((nRows-poolSize)//strides + 1)]
    remainder = [[[None for _ in range(nChannels)] for _ in range((nCols-poolSize)//strides + 1)] for _ in range((nRows-poolSize)//strides + 1)]
    for i in range((nRows-poolSize)//strides + 1):
        for j in range((nCols-poolSize)//strides + 1):
            for k in range(nChannels):
                for x in range(poolSize):
                    for y in range(poolSize):
                        out[i][j][k] += input[i*strides+x][j*strides+y][k]
                remainder[i][j][k] = str(out[i][j][k] % poolSize**2 % p)
                out[i][j][k] = str(out[i][j][k] // poolSize**2 % p)
    return Input, out, remainder


In [208]:
_, avg2d_1_out, avg2d_1_remainder = AveragePooling2DInt(24, 24, 6, 2, 2, avg2d_1_in)
avg2d_1_out[0][5]

['0', '0', '0', '18634872343567936', '0', '0']

In [209]:
avg2d_1_model = tf.keras.Model(inputs, model.layers[2].output)
avg2d_1_model.predict(X.reshape(1,28,28,1))[0][0][5]
#np.array(a).shape

1/1 [==============================] - 0s 29ms/step


array([0.        , 0.        , 0.        , 0.01863487, 0.        ,
       0.        ], dtype=float32)

In [224]:
conv2d_2_in = [[[int(avg2d_1_out[i][j][k]) for k in range(6)] for j in range(12)] for i in range(12)]
conv2d_2_weights = [[[[int(model.layers[4].weights[0][i][j][k][l]*1e18) for l in range(16)] for k in range(6)] for j in range(kernal_size)] for i in range(kernal_size)]
conv2d_2_bias = [int(model.layers[4].weights[1][i]*1e36) for i in range(16)]

In [225]:
_, conv2d_2_weights, conv2d_2_bias, conv2d_2_out, conv2d_2_remainder = Conv2DInt(12, 12, 6, 16, 5, 1, 10**18, conv2d_2_in, conv2d_2_weights, conv2d_2_bias)
conv2d_2_out[0][0]


['21888242871839275222246405745257275088548364400416034343693827815079926388811',
 '54715166402068534',
 '5358258363215520689',
 '21888242871839275222246405745257275088548364400416034343697437209211719010704',
 '4998874578075291060',
 '21888242871839275222246405745257275088548364400416034343697091145487710287657',
 '21888242871839275222246405745257275088548364400416034343687184498905292560795',
 '1298178389978459512',
 '1235018305609785554',
 '21888242871839275222246405745257275088548364400416034343689946682095862316167',
 '21888242871839275222246405745257275088548364400416034343696767755760320329366',
 '862256977227343804',
 '2358454384152901246',
 '21888242871839275222246405745257275088548364400416034343697835897056766275482',
 '4858408885536658813',
 '316357334054386690']

In [226]:
conv2d_2_model = Model(inputs, model.layers[4].output)
conv2d_2_model.predict(X.reshape(1,28,28,1))[0][0][0]



1/1 [==============================] - 0s 75ms/step


array([ -4.3763704 ,   0.05471502,   5.358259  ,  -0.7669772 ,
         4.9988766 ,  -1.1130408 , -11.0196905 ,   1.2981787 ,
         1.235018  ,  -8.2575035 ,  -1.4364306 ,   0.86225706,
         2.358455  ,  -0.36828965,   4.858408  ,   0.3163573 ],
      dtype=float32)

In [232]:
relu_2_in = [[[int(conv2d_2_out[i][j][k]) for k in range(16)] for j in range(8)] for i in range(8)]
relu_2_out = [[[str(relu_2_in[i][j][k]) if relu_2_in[i][j][k] < p//2 else 0 for k in range(16)] for j in range(8)] for i in range(8)]
relu_2_out[0][0]



[0,
 '54715166402068534',
 '5358258363215520689',
 0,
 '4998874578075291060',
 0,
 0,
 '1298178389978459512',
 '1235018305609785554',
 0,
 0,
 '862256977227343804',
 '2358454384152901246',
 0,
 '4858408885536658813',
 '316357334054386690']

In [231]:
relu_2_model = Model(inputs, model.layers[5].output)
relu_2_model.predict(X.reshape(1,28,28,1))[0][0][0]

1/1 [==============================] - 0s 25ms/step


array([0.        , 0.05471502, 5.358259  , 0.        , 4.9988766 ,
       0.        , 0.        , 1.2981787 , 1.235018  , 0.        ,
       0.        , 0.86225706, 2.358455  , 0.        , 4.858408  ,
       0.3163573 ], dtype=float32)

In [233]:
avg2d_2_in = [[[int(relu_2_out[i][j][k]) if int(relu_2_out[i][j][k]) < p//2 else int(relu_2_out[i][j][k]) - p for k in range(16)] for j in range(8)] for i in range(8)]


In [234]:
_, avg2d_2_out, avg2d_2_remainder = AveragePooling2DInt(8, 8, 16, 2, 2, avg2d_2_in)
avg2d_2_out[3][3]

['211313922284724448',
 '0',
 '0',
 '0',
 '256192674101796686',
 '60061032738268812',
 '773900356363521138',
 '0',
 '0',
 '474721562288850392',
 '2554274304934211113',
 '0',
 '278088946066706170',
 '0',
 '55919607932425602',
 '0']

In [235]:
avg2d_2_model = Model(inputs, model.layers[6].output)
avg2d_2_model.predict(X.reshape(1,28,28,1))[0][3][3]



1/1 [==============================] - 0s 23ms/step


array([0.2113139 , 0.        , 0.        , 0.        , 0.25619283,
       0.06006104, 0.77390015, 0.        , 0.        , 0.47472167,
       2.5542738 , 0.        , 0.27808905, 0.        , 0.05591962,
       0.        ], dtype=float32)

In [236]:
flatten_out = [avg2d_2_out[i][j][k] for i in range(4) for j in range(4) for k in range(16)]
flatten_out[100:120]

['0',
 '938984743660224675',
 '2067804480285152883',
 '1344624834281213250',
 '4091330354269761076',
 '332630101676133361',
 '0',
 '831309345064098638',
 '27453441204654108',
 '0',
 '0',
 '365910805562701939',
 '2497915349947138529',
 '1553310209583115223',
 '0',
 '0',
 '0',
 '3082250149731533625',
 '2085592566250439054',
 '1904644290955613972']

In [237]:
flatten_model = Model(inputs, model.layers[7].output)
flatten_model.predict(X.reshape(1,28,28,1))[0][100:120]

1/1 [==============================] - 0s 46ms/step


array([0.        , 0.938985  , 2.0678048 , 1.3446245 , 4.0913305 ,
       0.3326301 , 0.        , 0.8313095 , 0.02745353, 0.        ,
       0.        , 0.3659107 , 2.497915  , 1.5533102 , 0.        ,
       0.        , 0.        , 3.08225   , 2.085592  , 1.9046438 ],
      dtype=float32)

In [239]:
dense_in = [int(flatten_out[i]) if int(flatten_out[i]) < p//2 else int(flatten_out[i]) - p for i in range(256)]
dense_weights = [[int(model.layers[8].weights[0][i][j]*1e18) for j in range(10)] for i in range(256)]
dense_bias = [int(model.layers[8].weights[1][i]*1e36) for i in range(10)]

In [240]:
def DenseInt(nInputs, nOutputs, n, input, weights, bias):
    Input = [str(input[i] % p) for i in range(nInputs)]
    Weights = [[str(weights[i][j] % p) for j in range(nOutputs)] for i in range(nInputs)]
    Bias = [str(bias[i] % p) for i in range(nOutputs)]
    out = [0 for _ in range(nOutputs)]
    remainder = [None for _ in range(nOutputs)]
    for j in range(nOutputs):
        for i in range(nInputs):
            out[j] += input[i] * weights[i][j]
        out[j] += bias[j]
        remainder[j] = str(out[j] % n)
        out[j] = str(out[j] // n % p)
    return Input, Weights, Bias, out, remainder

In [241]:
_, dense_weights, dense_bias, dense_out, dense_remainder = DenseInt(256, 10, 10**18, dense_in, dense_weights, dense_bias)
dense_out

['21888242871839275222246405745257275088548364400416034343692457688804521142967',
 '21888242871839275222246405745257275088548364400416034343691320082705425112601',
 '605321176619278536',
 '979002222826562042',
 '21888242871839275222246405745257275088548364400416034343675491659713582475381',
 '21888242871839275222246405745257275088548364400416034343690582961252262862404',
 '21888242871839275222246405745257275088548364400416034343668568920446582535159',
 '14050346802623423396',
 '21888242871839275222246405745257275088548364400416034343690202069175191903095',
 '21888242871839275222246405745257275088548364400416034343696179033481887985342']

In [243]:
dense_model = Model(inputs, model.layers[-1].output)
dense_model.predict(X.reshape(1,28,28,1))[0]

1/1 [==============================] - 0s 81ms/step


array([ -5.7464976 ,  -6.884104  ,   0.60532194,   0.9790039 ,
       -22.712532  ,  -7.6212263 , -29.63527   ,  14.050348  ,
        -8.002118  ,  -2.0251534 ], dtype=float32)

In [248]:
in_json = {
    "in": X_in,
    "conv2d_weights": conv2d_1_weights,
    "conv2d_bias": conv2d_1_bias,
    "conv2d_out": conv2d_1_out,
    "conv2d_remainder": conv2d_1_remainder,
    "conv2d_re_lu_out": relu_1_out,
    "average_pooling2d_out": avg2d_1_out,
    "average_pooling2d_remainder": avg2d_1_remainder,
    "conv2d_1_weights": conv2d_2_weights,
    "conv2d_1_bias": conv2d_2_bias,
    "conv2d_1_out": conv2d_2_out,
    "conv2d_1_remainder": conv2d_2_remainder,
    "conv2d_1_re_lu_out": relu_2_out,
    "average_pooling2d_1_out": avg2d_2_out,
    "average_pooling2d_1_remainder": avg2d_2_remainder,
    "flatten_out": flatten_out,
    "dense_weights": dense_weights,
    "dense_bias": dense_bias,
    "dense_out": dense_out,
    "dense_remainder": dense_remainder
}

In [249]:
with open("mnist_input.json", "w") as f:
    json.dump(in_json, f)

In [160]:
layers = [14,6,16,10]

In [161]:
circuit_folder = "./golden_circuits/"
target_circom = "_".join(str(x) for x in layers) + '.circom'

command = ['circom', circuit_folder + target_circom, "--r1cs", "--wasm", "--sym", "-o", tmp_folder]
subprocess.run(command)

template instances: 23
non-linear constraints: 2558720
linear constraints: 0
public inputs: 0
private inputs: 21882 (15994 belong to witness)
public outputs: 10
wires: 2561275
labels: 5939891
Written successfully: ./tmp/14_6_16_10.r1cs
Written successfully: ./tmp/14_6_16_10.sym
Written successfully: ./tmp/14_6_16_10_js/14_6_16_10.wasm
Everything went okay


CompletedProcess(args=['circom', './golden_circuits/14_6_16_10.circom', '--r1cs', '--wasm', '--sym', '-o', './tmp/'], returncode=0)

In [271]:
circuit_folder = "./golden_circuits/"
target_circom = "mnist_test.circom"

command = ['circom', circuit_folder + target_circom, "--r1cs", "--wasm", "--sym", "-o", tmp_folder]
subprocess.run(command)

template instances: 30
non-linear constraints: 1965827
linear constraints: 0
public inputs: 0
private inputs: 23487 (18718 belong to witness)
public outputs: 1
wires: 1969848
labels: 4193424
Written successfully: ./tmp/mnist_test.r1cs
Written successfully: ./tmp/mnist_test.sym
Written successfully: ./tmp/mnist_test_js/mnist_test.wasm
Everything went okay


CompletedProcess(args=['circom', './golden_circuits/mnist_test.circom', '--r1cs', '--wasm', '--sym', '-o', './tmp/'], returncode=0)

In [277]:
2**22 - 2558720 * 2

-923136

In [250]:
# Gen Witness
json_folder = tmp_folder + target_circom[:-7] + "_js/"
wit_json_file = json_folder + "generate_witness.js"
wasm_file = json_folder + target_circom[:-7] + ".wasm"
input_path = "mnist_input.json"
wit_file = tmp_folder + "witness.wtns"

command = ['node', wit_json_file, wasm_file, input_path, wit_file]
subprocess.run(command)

CompletedProcess(args=['node', './tmp/28_6_16_10_js/generate_witness.js', './tmp/28_6_16_10_js/28_6_16_10.wasm', 'mnist_input.json', './tmp/witness.wtns'], returncode=0)

### Trusted Setup

In [251]:
ceremony_folder = "./tmp/big_ceremony/"
zkey_1 = ceremony_folder + 'test_0000.zkey'
veri_key = ceremony_folder + 'vk.json'

In [262]:
# Setup
digit = 19 # 2**22 = 4194304 2558720*2 = -3000 
tmp_folder = './tmp/'
ceremony_folder = tmp_folder + 'very_big_ceremony/'
os.makedirs(ceremony_folder, exist_ok=True)
ptau_1 = ceremony_folder + 'pot12_0000.ptau'

command = ['snarkjs', 'powersoftau', 'new', 'bn128', str(digit), ptau_1,'-v']
subprocess.run(command)

[DEBUG] snarkJS: tauG1: 100000
[DEBUG] snarkJS: tauG1: 200000
[DEBUG] snarkJS: tauG1: 300000
[DEBUG] snarkJS: tauG1: 400000
[DEBUG] snarkJS: tauG1: 500000
[DEBUG] snarkJS: tauG1: 600000
[DEBUG] snarkJS: tauG1: 700000
[DEBUG] snarkJS: tauG1: 800000
[DEBUG] snarkJS: tauG1: 900000
[DEBUG] snarkJS: tauG1: 1000000
[DEBUG] snarkJS: tauG2: 100000
[DEBUG] snarkJS: tauG2: 200000
[DEBUG] snarkJS: tauG2: 300000
[DEBUG] snarkJS: tauG2: 400000
[DEBUG] snarkJS: tauG2: 500000
[DEBUG] snarkJS: alphaTauG1: 100000
[DEBUG] snarkJS: alphaTauG1: 200000
[DEBUG] snarkJS: alphaTauG1: 300000
[DEBUG] snarkJS: alphaTauG1: 400000
[DEBUG] snarkJS: alphaTauG1: 500000
[DEBUG] snarkJS: betaTauG1: 100000
[DEBUG] snarkJS: betaTauG1: 200000
[DEBUG] snarkJS: betaTauG1: 300000
[DEBUG] snarkJS: betaTauG1: 400000
[DEBUG] snarkJS: betaTauG1: 500000
[DEBUG] snarkJS: Calculating First Challenge Hash
[DEBUG] snarkJS: Calculate Initial Hash: tauG1
[DEBUG] snarkJS: Initial hash: 0
[DEBUG] snarkJS: Initial hash: 341000
[DEBUG] sna

CompletedProcess(args=['snarkjs', 'powersoftau', 'new', 'bn128', '19', './tmp/very_big_ceremony/pot12_0000.ptau', '-v'], returncode=0)

Run this in terminal as asking random input for key gen
```bash
snarkjs powersoftau contribute ./tmp/very_big_ceremony/pot12_0000.ptau ./tmp/very_big_ceremony/pot12_0001.ptau --name="1st" -v
```

In [ ]:
command = ["snarkjs", "powersoftau", "contribute", ptau_1, ptau_2, "--name=1st", "-v"]

In [263]:
ptau_2 = ceremony_folder + 'pot12_0001.ptau'
ptau_3 = ceremony_folder + 'pot12_final.ptau'

command = ['snarkjs', 'powersoftau', 'prepare', 'phase2', ptau_2,ptau_3, '-v']
subprocess.run(command)

[DEBUG] snarkJS: Starting section: tauG1
[DEBUG] snarkJS: tauG1: fft 0 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 0 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 1 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 1 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 2 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 2 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 3 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 3 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 4 mix start: 0/2
[DEBUG] snarkJS: tauG1: fft 4 mix start: 1/2
[DEBUG] snarkJS: tauG1: fft 4 mix end: 0/2
[DEBUG] snarkJS: tauG1: fft 4 mix end: 1/2
[DEBUG] snarkJS: tauG1: fft  4  join: 4/4
[DEBUG] snarkJS: tauG1: fft 4 join  4/4  1/1 0/1
[DEBUG] snarkJS: tauG1: fft 5 mix start: 0/4
[DEBUG] snarkJS: tauG1: fft 5 mix start: 1/4
[DEBUG] snarkJS: tauG1: fft 5 mix start: 2/4
[DEBUG] snarkJS: tauG1: fft 5 mix start: 3/4
[DEBUG] snarkJS: tauG1: fft 5 mix end: 0/4
[DEBUG] snarkJS: tauG1: fft 5 mix end: 1/4
[DEBUG] snarkJS: tauG1: fft 5 mix end: 2/4
[DEBUG] snarkJS: tauG1: fft 5 m

CompletedProcess(args=['snarkjs', 'powersoftau', 'prepare', 'phase2', './tmp/very_big_ceremony/pot12_0001.ptau', './tmp/very_big_ceremony/pot12_final.ptau', '-v'], returncode=0)

In [265]:
r1cs_path = tmp_folder + "_".join(str(x) for x in layers) + ".r1cs"
zkey_1 = ceremony_folder + 'test_0000.zkey'
command = ['snarkjs', 'groth16', 'setup', r1cs_path, ptau_3, zkey_1]
subprocess.run(command)

[ERROR] snarkJS: circuit too big for this power of tau ceremony. 2558720*2 > 2**19


CompletedProcess(args=['snarkjs', 'groth16', 'setup', './tmp/28_6_16_10.r1cs', './tmp/very_big_ceremony/pot12_final.ptau', './tmp/very_big_ceremony/test_0000.zkey'], returncode=0)

In [270]:
2**23 - 2558720 * 2

3271168

In [252]:
command = ['snarkjs', 'groth16', 'prove',zkey_1, wit_file, tmp_folder+'proof.json', tmp_folder+'public.json']
subprocess.run(command)

[ERROR] snarkJS: Error: Invalid witness length. Circuit: 30223, witness: 2561275
    at groth16Prove$1 (/home/guy1m0/.nvm/versions/node/v21.6.1/lib/node_modules/snarkjs/build/cli.cjs:5496:15)
    at async Object.groth16Prove [as action] (/home/guy1m0/.nvm/versions/node/v21.6.1/lib/node_modules/snarkjs/build/cli.cjs:8339:36)
    at async clProcessor (/home/guy1m0/.nvm/versions/node/v21.6.1/lib/node_modules/snarkjs/build/cli.cjs:304:27)


CompletedProcess(args=['snarkjs', 'groth16', 'prove', './tmp/big_ceremony/test_0000.zkey', './tmp/witness.wtns', './tmp/proof.json', './tmp/public.json'], returncode=1)

## Benchmark for 784_6_16_120_84_10 CNN Model

In [281]:
layers = [28, 6, 16, 120, 84, 10]

inputs = tf.keras.layers.Input(shape=(28,28,1))
out = tf.keras.layers.Conv2D(layers[1],5, use_bias = True)(inputs)
out = tf.keras.layers.ReLU()(out)
out = tf.keras.layers.AveragePooling2D()(out)
out = tf.keras.layers.Conv2D(layers[2],5, use_bias = True)(out)
out = tf.keras.layers.ReLU()(out)
out = tf.keras.layers.AveragePooling2D()(out)
out = tf.keras.layers.Flatten()(out)
out = tf.keras.layers.Dense(layers[3], activation = 'relu')(out)
out = tf.keras.layers.Dense(layers[4], activation = 'relu')(out)
out = tf.keras.layers.Dense(layers[5])(out)

model = tf.keras.Model(inputs, out)

In [66]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='relu'),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dense(84, activation='relu'),
    tf.keras.layers.Dense(10)  # Assuming 10 classes
])

In [279]:
model_path = "../../models/"
arch_folder = "input-conv2d-conv2d-dense-dense-dense/"
model_name = "784_6_16_120_84_10.h5"
model_in_path = model_path+arch_folder+model_name

model.load_weights(model_in_path)
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

Model: "model_49"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_61 (Conv2D)          (None, 24, 24, 6)         156       
                                                                 
 re_lu_40 (ReLU)             (None, 24, 24, 6)         0         
                                                                 
 average_pooling2d_60 (Aver  (None, 12, 12, 6)         0         
 agePooling2D)                                                   
                                                                 
 conv2d_62 (Conv2D)          (None, 8, 8, 16)          2416      
                                                                 
 re_lu_41 (ReLU)             (None, 8, 8, 16)          0         
                                                          

In [280]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 0s - loss: 1.1180 - accuracy: 0.9866 - 318ms/epoch - 1ms/step

Test accuracy: 0.9865999817848206


In [3]:
import tensorflow as tf

2024-02-06 01:48:20.485780: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 01:48:20.506489: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 01:48:20.506510: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 01:48:20.507050: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 01:48:20.510610: I tensorflow/core/platform/cpu_feature_guar

In [5]:
layers = [14, 5, 11, 84, 10, 3]

inputs = tf.keras.layers.Input(shape=(layers[0],layers[0],1))
out = tf.keras.layers.Conv2D(layers[1],layers[-1], use_bias = True)(inputs)
out = tf.keras.layers.ReLU()(out)
out = tf.keras.layers.AveragePooling2D()(out)
out = tf.keras.layers.Conv2D(layers[2],layers[-1], use_bias = True)(out)
out = tf.keras.layers.ReLU()(out)
out = tf.keras.layers.AveragePooling2D()(out)
out = tf.keras.layers.Flatten()(out)
out = tf.keras.layers.Dense(layers[3])(out)
out = tf.keras.layers.Dense(layers[4])(out)

model = tf.keras.Model(inputs, out)
model.summary()
model.save("test.h5")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 14, 14, 1)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 5)         50        
                                                                 
 re_lu_1 (ReLU)              (None, 12, 12, 5)         0         
                                                                 
 average_pooling2d_1 (Avera  (None, 6, 6, 5)           0         
 gePooling2D)                                                    
                                                                 
 conv2d_3 (Conv2D)           (None, 4, 4, 11)          506       
                                                                 
 re_lu_2 (ReLU)              (None, 4, 4, 11)          0         
                                                             

/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [373]:
circuit_folder = "./golden_circuits/"
target_circom = "circuit.circom"

command = ['circom', circuit_folder + target_circom, "--r1cs", "--wasm", "--sym", "-o", tmp_folder]
subprocess.run(command)

template instances: 31
non-linear constraints: 482198
linear constraints: 0
public inputs: 0
private inputs: 8789 (7606 belong to witness)
public outputs: 3
wires: 487023
labels: 1056500
Written successfully: ./tmp/circuit.r1cs
Written successfully: ./tmp/circuit.sym
Written successfully: ./tmp/circuit_js/circuit.wasm
Everything went okay


CompletedProcess(args=['circom', './golden_circuits/circuit.circom', '--r1cs', '--wasm', '--sym', '-o', './tmp/'], returncode=0)

In [368]:
2**20 - 490000 * 2

68576

In [297]:
2**22 - 2597800 * 2

-1001296